In [66]:
#cycle gan does unpaired learning

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

import os
import glob
import time
import matplotlib.pyplot as plt

AUTOTUNE = tf.data.AUTOTUNE

In [67]:
#create a list of image file paths
# https://cs230.stanford.edu/blog/datapipeline/#best-practices

DATA_DIR = r"\\shelter\Kyu\unstain2stain\unstain2stain_tile\train"
train_he = sorted(glob.glob(os.path.join(*[DATA_DIR, 'HE\OTS_14684_2_he\*.png'])))
train_unstained = sorted(glob.glob(os.path.join(*[DATA_DIR,'Unstained\OTS_14684_2\*.png'])))
val_he = sorted(glob.glob(os.path.join(*[DATA_DIR, 'HE\OTS_14684_3_he\*.png'])))
val_unstained = sorted(glob.glob(os.path.join(*[DATA_DIR, 'Unstained\OTS_14684_3\*.png'])))

In [68]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 1024
IMG_HEIGHT = 1024


In [69]:
#image pre-process
def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image

# normalizing the images to [-1, 1]
def normalize(image,label):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

def random_jitter(image,label):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image)

  # random mirroring
  image = tf.image.random_flip_left_right(image)

  return image

In [70]:
def preprocess_image_train(image):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [71]:
def preprocess_image_test(image):
  image = normalize(image)
  return image

In [74]:
#copied from kyu
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    # image = cv2.imread(image_path.decode('UTF-8'))
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMG_WIDTH, IMG_WIDTH])
        # image[image==13]=0
        image = tf.where(tf.equal(image, 13), tf.zeros_like(image), image)
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMG_WIDTH, IMG_WIDTH])
        # image = image / 127.5 - 1 #[-1 1]
        image = image/255 #[0 1]
    return image

def load_data(image_list):
    image = read_image(image_list)
    #mask = read_image(mask_list)
    return image

def data_generator(image_list):
    dataset = tf.data.Dataset.from_tensor_slices(image_list,labels)
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    # dataset = dataset.map(lambda x,y: tf.numpy_function(load_data, [x,y], Tout=tf.uint8))
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    # dataset = dataset.prefetch(1) #added this to always prefetch 1 batch
    return dataset

In [75]:
unstained_dataset = data_generator(train_unstained)
he_dataset = data_generator(train_he)

NameError: name 'labels' is not defined

In [57]:
#feed into a model

In [58]:
dataset, metadata = tfds.load('cycle_gan/horse2zebra',
                              with_info=True, as_supervised=True)

train_horses, train_zebras = dataset['trainA'], dataset['trainB']

In [76]:
train_horses

<PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [79]:
def preprocess_image_train(image, label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [81]:
def random_jitter(image):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image)

  # random mirroring
  image = tf.image.random_flip_left_right(image)

  return image

In [83]:
# normalizing the images to [-1, 1]
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

In [84]:
train_horses = train_horses.cache().map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

In [85]:
sample_horse = next(iter(train_horses))

InvalidArgumentError: assertion failed: [Need value.shape >= size, got ] [286 286 3] [1024 1024 3]
	 [[{{node random_crop/Assert/Assert}}]] [Op:IteratorGetNext]

In [64]:
unstained_dataset = unstained_dataset.cache().map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

he_dataset = he_dataset.cache().map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

ValueError: in user code:

    C:\Users\labadmin\AppData\Local\Temp\ipykernel_14728\24941931.py:2 preprocess_image_train  *
        image = random_jitter(image)
    C:\Users\labadmin\AppData\Local\Temp\ipykernel_14728\2093225660.py:20 random_jitter  *
        image = random_crop(image)
    C:\Users\labadmin\AppData\Local\Temp\ipykernel_14728\2093225660.py:3 random_crop  *
        cropped_image = tf.image.random_crop(
    C:\ProgramData\Anaconda3\envs\wsi_analysis39\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    C:\ProgramData\Anaconda3\envs\wsi_analysis39\lib\site-packages\tensorflow\python\ops\random_ops.py:402 random_crop
        math_ops.reduce_all(shape >= size),
    C:\ProgramData\Anaconda3\envs\wsi_analysis39\lib\site-packages\tensorflow\python\ops\math_ops.py:1817 wrapper
        return fn(x, y, *args, **kwargs)
    C:\ProgramData\Anaconda3\envs\wsi_analysis39\lib\site-packages\tensorflow\python\ops\gen_math_ops.py